<h3>importing </h3>

In [1]:
#uncomment below line to install libraries needed for this project

# !pip install hazm gdown numpy pandas

In [2]:
import numpy as np
import pandas as pd
import hazm as hz
import gdown

<h3>downloading files and dependacies</h3>

In [3]:
#uncomment the lines below to dowmload the files needed for this project

# gdown.download(id = "1E7HUmSArVP6LmyaOjByka2dYME_iBS47",output="books_train.csv")
# gdown.download(id = "1nB61mWEy1vYcZyyPwiifQBwz_Hgo1EJc",output="books_test.csv")
# gdown.download(id = "1D40HFl-gClrxaD606SVag4dfA_OD9Qlt",output="spec_chars.txt")
# gdown.download(id = "17xQuS6Y0xgT80aGcTjLrR8CpUGN85QsR",output="words_to_delete.txt")

<h3>reading the data frames</h3>

<p dir="rtl" lang="fa" align="center">اولین قدم در اجرای این پروژه، خواندن فایل های تست و ترین با کمک کتابخانه پانداز می‌باشد</p>

In [4]:
books_test = pd.read_csv("books_test.csv")

books_train = pd.read_csv("books_train.csv")

<h3>phase 1: preprocessing</h3>

<p dir="rtl" lang="fa" align="center">در مرحله بعد، دو فایل با کمک پانداز بارگذاری میشود. فایل اول، شامل حروف اضافه، حروف ربط، ضمایر و برخی صفات اشاره است. فایل دوم شامل علائم نگارشی، اعداد و برخی از یونیکد های خاص فارسی است. علت تغییر آرکومان های delimiter و qutochar، واکنش خاصی است که پانداز به علائم کاما و دابل کوتیشن نشان میدهد</p>

In [5]:
words_to_delete = list(pd.read_csv(
    "words_to_delete.txt"
    ,header=None
    ,names=["dels"]
    )["dels"])
spec_chars = list(pd.read_csv(
    "spec_chars.txt",
    header=None,
    names=['chr'],
    delimiter=" ",
    quotechar= " "
    )['chr'])


<p dir="rtl" lang="fa" align="center">دو آبجکت نرمالایزر و توکنایزر برای استفاده از متد های این دو کلاس از کابخانه هضم ساخته میشود.</p>

In [6]:
norm_obj = hz.Normalizer()
token_obj =  hz.WordTokenizer()

<p dir="rtl" lang="fa" align="center">برویم سراغ اولین حرکت جدی این پروژه! تابع کلین تکست، وظیفه انجام اکشن های پری پراسس را بر روی استرینگ ورودی به این تابع دارد. در ابتدا از متد نرمالایز هضم استفاده میشود. این متد تا حدود خوبی متن را شبیه به متن نوشتاری رسمی میکند، با انجام کارهایی مانند دور ریختن اسپیس های تکراری،از بین بردن اعراب حروف، تبدیل اعداد به اعداد فارسی، درست کردن نیم فاصله افعال و ... که در روند پردازش متن کمک شایانی میکند. سپس از طربق متد replace نیم فاصله را با فاصله جایگذاری میکنیم تا پیشوند ها و پسوند های اضافی که پیشتر بار گذاری کرده بودیم را حذف کنیم. در مرحله بعد، با استفاده از متد توکنایز کتابخانه هضم، کلمات را از هم جدا میکنیم و در قالب لیست نگه میداریم. متد توکنایز با بررسی جمله، کلمات، اعداد و علائم را از هم جدا کرده و در قالب لیست بر میگرداند. حال کلماتی را که در لیست <<کلمات قابل حذف>> نیستند را شناسایی کرده و به هم میچسبانیم. در مرحله بعد، روی استرینگ ایجاد شده حرکت میکنیم و هر حرفی که جزو حروف فایل علائم نبود را انتخاب و دوباره به هم میچسبانیم. در مرحله آخر، متن تمیز شده را بر میگردانیم. به کد این بخش توجه کنید: </p>

In [7]:
def clean_text(text : str) -> str:
  text = norm_obj.normalize(text).replace("‌"," ")
  tokenized_text_list = token_obj.tokenize(text)
  semi_final_text = " ".join([x for x in tokenized_text_list if x not in words_to_delete])
  final_text = "".join([x for x in semi_final_text if x not in spec_chars])
  return final_text

<p dir="rtl" lang="fa" align="center"> حال در این بخش ابتدا نگاهی کلی به دیتا فریم تست می اندازیم تا ببینیم آیا داده گمشده داریم یا خیر، که متوجه می شویم نداریم. سپس اعمال پری پراسس را روی ستون توضیحات این دیتا فریم، با کمک متد اپلای پانداز (که یک تابع را به طور همزمان روی همه سطر های یک دیتا فریم پیاده میکند) پیاده میکنیم. در مرحله آخر هم ابتدای ستون توضیحات را یک نگاه کلی می اندازیم.</p>

In [8]:
books_train.info(verbose=True,show_counts=True)

books_train["description"] = books_train["description"].apply(clean_text)
books_train["description"].head(10)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2550 entries, 0 to 2549
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        2550 non-null   object
 1   description  2550 non-null   object
 2   categories   2550 non-null   object
dtypes: object(3)
memory usage: 59.9+ KB


0     ساختار نظریه جامعه شناسی ایران  نوشته ابوالفض...
1     جامعه فرهنگ کانادا  مجموعه کتاب  جامعه فرهنگ ...
2    پرسش مختلفی زندگی شخصیت امام مهدی  عج  اذهان م...
3     موج دریا  قلم مهری ماهوتی     تصویرگری عاطفه ...
4     پرسش غرب  قلم دکتر اسماعیل شفیعی سروستانی  نو...
5     خارج خط  مجموعه داستان کوتاهی نوشته محمود راج...
6     لاک صورتی  نوشته جلال آل احمد      نویسنده فع...
7    راه بسیار زیادی سرمایه گذاری دنیا وجود دارد  ی...
8    شکل گیری رشد روز افزون دهکده جهانی  بشر بیش زم...
9     رویکردی جدید اختلالات مصرف مواد مشاوره اعتیاد...
Name: description, dtype: object

<p dir="rtl" lang="fa" align="center">در این بخش همه اعمال بخش قبل را روی دیتا فریم تست پیاده میکنیم. نکته مهم اینکه این دیتافریم نیز مقادیر گمشده نداشت.</p>

In [9]:
books_test.info(verbose=True,show_counts=True)

books_test["description"] = books_test["description"].apply(clean_text)
books_test.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 450 entries, 0 to 449
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        450 non-null    object
 1   description  450 non-null    object
 2   categories   450 non-null    object
dtypes: object(3)
memory usage: 10.7+ KB


,title,description,categories
0,کآشوب,کآشوب بیست سه روایت روضه هایی زندگی کنیم ه...,داستان کوتاه
1,داستان‌های برق‌آسا,داستان برق آسا نام مجموعه داستان هایی گردآور...,داستان کوتاه
2,بحثی درباره مرجعیت و روحانیت,مجموعه مقالات بحثی مرجعیت روحانیت شامل مقالا...,کلیات اسلام
3,قلعه‌ی حیوانات,قلعه حیوانات جورج اورول گروهی حیوانات اهلی ...,رمان
4,قصه ما مثل شد (۱),قصه مثل شد یک مجموعه کتاب جلدی است محمد میر...,داستان کودک و نوجوانان


<h3>phase2: solving the problem</h3>

<p dir="rtl" lang="fa" align="center">وقت حل مسئله است! ابتدایی ترین قدم، فیلتر کردن ستون توضیحات دیتا فریم تست بر اساس کتگوری میباشد. این عمل به این دلیل است که بعدتر بتوانیم کلمات مربوط به هر کتگوری را در BoW راحت تر دسته بندی کنیم. تابع زیر مسئول این کار میباشد. این تابع لیست کتگوری ها و دیتا فریم مورد بررسی را به عنوان آرگومان دریافت میکند. ابتدا یک لیست خالی برای نگه داشتن سری (منظور pandas series  است، چرا که ما فقط با ستون توضیحات کار داریم و هر ستون یک دیتا فریم در پانداز، یک سری میباشد) های فیلتر شده درست میکند. سپس در هر مرحله، با استفاده از متد loc پانداز، همه سطر های توضیحاتی که یک کتگوری دارند به هم چسبانده و در قالب پانداز سری برگردانده شده، و پس از آن به لیست ساخته شده اضافه میشود. این کار برای همه کتگوری های پاس داده شده به تابع انجام شده، سپس لیست مذکور توسط تابع ریترن میشود. </p>

In [10]:
def make_cat_filt(categories : list, books_train : pd.DataFrame) -> list:
    category_filtered_list = list()
    for x in categories:
        category_filtered_list.append(
            books_train["description"].loc[books_train["categories"] == x]
            )
    return category_filtered_list

<p dir="rtl" lang="fa" align="center">در این مرحله، باید از لیست سری های فیلتر شده مان استفاده کنیم و BoW را بسازیم. در ابتدا یک لیست خالی به نام series_to_concat میسازیم تا سری هایی که هر کدام نشان دهنده کلمات یک کتگوری هستند را در آن نگه داشته و در آخرین مرحله با هم ترکیب کنیم. در خط بعد یک حلقه فور داریم که عمل شمردن کلمات را برای هر کتگوری انجام میدهد. پیش از بررسی خط بعد، باید توضیح بدهم که هدف این است که برای هر کتگوری، یک سری بسازیم که ایندکس های آن، کلمات دیده شده در هر کتگوری و مقادیر آن، تعداد تکرار این کلمات می باشد. حال این کار را چگونه انجام میدهیم؟ در ابتدا برای هر عضو لیست فیلتر شده، از متد .str.split() استفاده میکنیم که وظیفه آن، تبدیل متن هر خانه پانداز سری به لیستی از کلمات آن متن میباشد. سپس متد .explode() همه این لیست های ایجاد شده را به هم چسبانده و در قالب یک سری بسیار بلند بر میگرداند که هر خانه آن، یک کلمه است. سپس متد .value_counts() تعداد تکرار هر کلمه را در این سری بسیار بلند شمرده و در قالب یک سری دیگر بر میگرداند که ایندکس آن، کلمات و مقادیر آن تعداد تکرار هر کلمه میباشد. در نهایت  نام سری ایجاد شده را به نام کتگوری مورد نظر تغییر داده و به لیست series_to_concat اضافه میکنیم. در خط بعد و پس از انجام مراحل بالا برای همه کتگوری ها، از متد .concat() پانداز استفاده میکنیم که این سری های ساخته شده را به یک دیتافریم تبدیل کنیم. به این متد در ابتدا لیست series_to_concat پاس داده میشود، سپس نوع اضافه شدن سری ها مشخص میشود که در اینجا اجتماع همه ایندکس ها قرار داده شده. و در مرحله بعد با قرار دادن axis= 1 به متد میفهمانیم که در نظر داریم این سری ها را به شکل ستونی به هم بچسبانیم. سپس  متد fillna(0) استفاده میشود، چرا که در دیتافریم ساخته شده، کتگوری هایی که ایندکسی را نداشته اند، برای آن مقدار nan قرار میدهد که باید این مقدار با مقدار 0 جایگزین شود (چراکه نبود ایندکس به معنی نبود آن کلمه در کتگوری مورد نظر است) در نهایت تایپ دیتافریم به عدد صحیح تغیر پیدا کرده و تابع، BoW ساخته شده را بر میگرداند</p>

In [11]:
def make_BoW(category_filtered_list : list, categories : list) -> pd.DataFrame:
    series_to_concat = list()
    for x in range(len(category_filtered_list)):
        series_to_concat.append(category_filtered_list[x].str.split().explode().value_counts().rename(categories[x]))
    BoW = pd.concat(series_to_concat,join="outer",axis = 1).fillna(0).astype(int)
    return BoW

<p dir="rtl" lang="fa" align="center">حال از توابع ایجاد شده استفاده کرده و BoW را تشکیل میدهیم. در ابتدا تمام کتگوری را ها با استفاده از متد unique پانداز (که کلماتی که در یک سری دیده شده اند را بر میگرداند) استخراج کرده و در قالب لیست ذخیره میکنیم. سپس category_filtered_list و BoW  را با توابع توضیح داده شده میسازیم. در نهایت نگاهی به ابتدای BoW می اندازیم. </p>

In [12]:
categories = list(books_train['categories'].unique())
category_filtered_list = make_cat_filt(categories, books_train)
BoW = make_BoW(category_filtered_list, categories)
BoW.head(15)

,جامعه‌شناسی,کلیات اسلام,داستان کودک و نوجوانان,داستان کوتاه,مدیریت و کسب و کار,رمان
description,,,,,,
است,1375,1005,673,1131,1286,1690
کتاب,838,598,439,477,968,748
اجتماعی,458,50,3,40,39,54
جامعه,453,79,5,34,35,51
کند,388,156,229,341,388,727
خود,360,213,93,268,519,473
یک,342,143,275,485,600,806
شود,298,156,134,226,243,500
رسانه,295,5,0,4,12,2


<p dir="rtl" lang="fa" align="center">همانطور که مشاهده میکنید، حروف اضافه و علائم و اعداد از بین رفته اند، اما هنوز برخی کلمات مانند «است» در دیتافریم وجود دارند که به نظر میرسد خیلی به درد بخور نیستند! برای آنها در بخش امتیازی تصمیم میگیریم.</p>
<hr></hr>
<p dir="rtl" lang="fa" align="center">تا به اینجا بنظر پیشرفت خیلی خوبی داشتیم! حال نوبت آن رسیده است که مقدمات محاسبه احتمال را فراهم کنیم. در ابتدا باید احتمال هر کتگوری را توسط تابع زیر محاسبه کنیم. این تابع category_filtered_list و books_train و categories را به عنوان آرگومان دریافت کرده و یک لیست خالی برای نگهداری تعداد کتاب های هر کتگوری میسازد. سپس  تعداد کتاب های هر کتگوری را به صورت جدا به وسیله حرکت روی category_filtered_list پیدا کرده و به لیست اضافه میکند (شمردن تعداد توضیحات برابر با شمردن تعداد کتاب ها است). پس از آن، تعداد کل کتاب ها را شمرده در مرحله بعد لیست نگهداری تعداد را به پانداز سری تبدیل میکند که بتوانیم روی آن عملیات جبری انجام دهیم. سپس سری مورد نظر را به تعداد کل کتاب های تقسیم میکند (هر خانه سری به این مقدار تقسیم میشود) و ایندکس های آن را کتگوری هایمان قرار داده و آن را بر میگرداند.   </p>

In [13]:
def calc_prior(category_filtered_list : list, books_train : pd.DataFrame , categories : list) -> pd.Series:
    categories_books_count = []
    for x in category_filtered_list:
        categories_books_count.append(len(x))
    sum_of_books = len(books_train.index)
    categories_books_count = pd.Series(categories_books_count)
    prior = categories_books_count / sum_of_books
    prior.index = categories
    return prior

<p dir="rtl" lang="fa" align="center">احتمال پیشینمان را با تابع توضیح داده شده محاسبه و سپس مشاهده میکنیم.</p>

In [14]:
prior = calc_prior(category_filtered_list, books_train, categories)
prior

جامعه‌شناسی               0.166667
کلیات اسلام               0.166667
داستان کودک و نوجوانان    0.166667
داستان کوتاه              0.166667
مدیریت و کسب و کار        0.166667
رمان                      0.166667
dtype: float64

<p dir="rtl" lang="fa" align="center">در اینجا بر حسب اتفاق، همه کتگوری ها احتمال برابر داشتند!</p>
<hr></hr>
<p dir="rtl" lang="fa" align="center">حال میخواهیم  کلمات موجود در بخش توضیحات دیتا فریم تست را در قالب یک سری که در هر خانه آن لیست کلمات توضیحات هر کتاب قرار دارد داشته باشیم. دوباره از متد str.split استفاده میکنیم و برای اینکه یک کلمه تکراری نباشد، از متد unique استفاده میکنیم.در لامبدا اکسپرشن نوشته شده، x هر سطر از سری است که یک لیست میباشد، و به این معنی است که 
unique مقدار هر سطر را به عنوان آرگومان دریافت کرده و مقدار بازگشتی اش را در همان سطر ذخیره میکند.  </p>

In [15]:
def make_books_test_desc(books_test : pd.DataFrame) -> pd.Series:
    books_test_desc = books_test["description"].str.split()
    books_test_desc = books_test_desc.apply(lambda x: pd.unique(x))
    return books_test_desc

<p dir="rtl" lang="fa" align="center">متغیر books_test_desc را با تابع بالا ساخته و نگاهی به ابتدای آن می اندازیم.</p>


In [16]:
books_test_desc = make_books_test_desc(books_test)
books_test_desc.head(5)

0    [کآشوب, بیست, سه, روایت, روضه, هایی, زندگی, کن...
1    [داستان, برق, آسا, نام, مجموعه, هایی, گردآوری,...
2    [مجموعه, مقالات, بحثی, مرجعیت, روحانیت, شامل, ...
3    [قلعه, حیوانات, جورج, اورول, گروهی, اهلی, است,...
4    [قصه, مثل, شد, یک, مجموعه, کتاب, جلدی, است, مح...
Name: description, dtype: object

<h3>phase2-1: without additive smoothing</h3>

<p dir="rtl" lang="fa" align="center">امیدوارم تا به اینجای کار از برنامه لذت برده باشید! در این مرحله قصد داریم دیتا فریم BoW را تبدیل به دیتا فریمی کنیم که رو به روی هر کلمه، احتمال آن کلمه به شرط کتگوری آن کلمه آمده باشد (بدون ادیتیو اسموتینگ). این عمل در تسریع محاسبات کمک شایانی میکند. احتمال هر کلمه به شرط کتگوری را هم طبق فرمول به شکل تقسیم تعداد تکرار آن کلمه در کتگوری بر تعداد کل کلمات کتگوری تعریف میکنیم. در اینجا با استفاده از قابلیت های پانداز و قرار دادن axis = 0 در هنگام جمع ، کل مقادیر هر ستون با هم جمع شده و در نهایت یک سری با ایندکس کتگوری ها و مقدار تعداد کلمات کتگوری به ما بر میگرداند که سپس با  تقسیم همه سطر های دیتا فریم بر این سطر تشکیل شده (این بار اکسیس را یک قرار میدهیم تا این عمل انجام شود) دیتا فریم مد نظر تشکیل میشود و آن را بر میگردانیم.</p>

In [17]:
def make_prob_df(df : pd.DataFrame) -> pd.DataFrame:
    final_df = df.copy()
    observ_cnt = df.sum(axis=0)
    final_df = final_df.div(observ_cnt,axis=1)
    return final_df

<p dir="rtl" lang="fa" align="center">پیش از آنکه مرحله آخر این بخش را پیاده سازی کنیم، لازم است به پرسش دوم پاسخ بدهیم (نگران نباشید جلوتر به پرسش اول هم میرسیم!). احتمال هر کلمه به شرط کتگوری مد نظر، مقداری کوچک میشود. حال فرض کنید به تعداد کلمات یک پاراگراف، از این اعداد کوچک در هم ضرب کنیم. احتمال آنقدر کم میشود که ممکن است در محاسبات کامپیوتری دچار خطا بشویم. با لگاریتم گرفتن از این مقادیر و جمع آنها، با اعدادی از اردر های نه چندان بزرگ و منفی سر و کار خواهیم داشت بعلاوه اینکه بین آنها هم جمع قرار میگیرد، بنابراین نیاری نیست نگران خیلی بزرگ شدن مقدار نهایی هم باشیم و مشکل خطای محاسبات اینگونه رفع میشود.</p>
<hr></hr>
<p dir="rtl" lang="fa" align="center">حال با توضیحات داده شده برویم سراغ اصل  مطلب! تابع زیر لیست کلمات توضیحات کتاب بعلاوه دیتا فریم احتمالات و سری احتمال پیشین را به عنوان آرگومان میپذیرد. سپس در ابتدا، بین کلمات کتاب و کلمات BoW با کمک متد intersection اشتراک میگیرد (این کار به این دلیل است که اگر کلمه ای در BoW نباشد، به ناچار در محاسبات، احتمال هر کتگوری را برای آن یکسان در نظر میگیریم. این هم ارز است با اینکه این کلمه را به کلی در نظر نگیریم و همان ابتدا از کلمات مورد محاسبه بیرون بریزیم). در مرحله بعد، زیر دیتافریمی از دیتافریم احتمالات فقط شامل کلمات متن میسازیم تا در محاسبات آینده صرفه جویی شود. سپس متد log  از کتابخانه نامپای را روی دیتا فریم اعمال میکنیم (آرگومان های آن برای این است که لگاریتم احتمالات صفر را برابر با صفر قرار دهد و ارور دریافت نکنیم. برابر با صفر قرار دادن این کلمات  هم ارز با ندید گرفتن آنها در محاسبات مربوط به بیز ساده است). در مرحله بعد، با جمع مقادیر هر ستون، به مقدار مورد نظر بیز ساده برای هر کتگوری میرسیم. سپس، سری به دست آمده که شامل کتگوری ها به عنوان کلید و احتمال متن به شرط کتگوری به عنوان مقدار میباشد را در سری احتمال پیشین ضرب میکنیم. مرحله آخر هم که تقسیم همه مقادیر نهایی بر احتمال متن است را نادیده میگیریم چرا که در مقایسه این احتمالات تاثیری ندارد. در نهایت ایندکسی که بیشترین مقدار را در سری probability دارد بر میگردانیم. این ایندکس در واقع کتگوری تخمین زده شده برای متن مورد نظر است. </p>

In [18]:

def cal_prob(text : list, prob_bow : pd.DataFrame, prior : pd.Series):
  existing_words = prob_bow.index.intersection(text)
  words_df = prob_bow.loc[existing_words]
  log_words = pd.DataFrame(np.log(
    words_df
    ,out= np.zeros_like(words_df)
    ,where=(words_df != 0)))
  likelihood = log_words.sum(axis = 0)
  probability = likelihood * prior
  return probability.idxmax()

<p dir="rtl" lang="fa" align="center">توابع توضیح داده شده را بر روی books_test_desc که شامل کلمات هر کتاب به شکل لیست است با کمک لامبدا اکسپرشن اپلای میکنیم. لامبدا اکسپرشن هم به این معنی است که هر سطر این دیتا فریم را به عنوان آرگومان اول که لیست کلمات است به تایع cal_prob بده و بقیه آرگومان ها را هم  prob_bow , prior قرار بده. سپس نتایج به دست آمده را با کتگوری های واقعی مقایسه میکنیم. نتیجه را ببینیم:</p>

In [19]:
prob_bow = make_prob_df(BoW)
categories_list_noadt = books_test_desc.apply(lambda x: cal_prob(x,prob_bow,prior))
(categories_list_noadt == books_test["categories"]).value_counts()

False    441
True       9
Name: count, dtype: int64

<p dir="rtl" lang="fa" align="center">مشاهده میکنیم که یک فاجعه ملی رخ داد! حال وقت آن است که به پرسش اول پاسخ دهیم. ندید گرفتن هر کلمه با احتمال صفر در کتگوری به این معنا است که در فرمول بیز ساده، احتمال آن را یک در نظر بگیریم، به عبارت دیگر ما با این کار احتمال این کلمه را از کمترین مقدار ممکن به بیشترین مقدار ممکن میبریم! پیامد این عمل آن است که هر کتگوری که صفر بیشتری داشته باشد هر دفعه برنده میشود. چاره این اشتباه فرمول ادیتیو اسموتینگ است که جلوتر به آن میپردازیم. ولی اگر بخواهیم بدون این فرمول کمی نتایج را بهتر کنیم، لازم است ارزش یک را تا جای ممکن در محاسبات خود کاهش دهیم. این کار چگونه ممکن است؟ درست حدس زدید، با اسکیل کردن مقادیر احتمال ها. حال بیاید بدون ورود به ادیتیو اسموتینگ، با اسکیل 2000، یک بار دیگر نتیجه را بررسی کنیم:  </p>

In [20]:
categories_list_noadt_2 = books_test_desc.apply(lambda x: cal_prob(x,prob_bow * 2000,prior))
(categories_list_noadt_2 == books_test["categories"]).value_counts()

False    235
True     215
Name: count, dtype: int64

<p dir="rtl" lang="fa" align="center">خیلی بهتر شد، اما هنوز هم کافی نیست. برای باز هم بهتر شدن، بروید سراغ بخش بعد!</p>

<h3>phase 2-2: with additive smoothing</h3>

<p dir="rtl" lang="fa" align="center">ادیتیو سموتینگ در واقع به همه احتمال های صفر، بر اساس مجموع مقادیر دسته بندی و تعداد مقادیر یکتا، یک احتمال خیلی کوچک میدهد. تابع زیر این عمل را بر اساس فرمول ادیتیو اسموتینگ روی دیتا فریم مورد نظر اعمال میکند. در ابتدا دیتا فریم مورد نظر را به عنوان آرگومان دریافت میکند. سپس مقادیر یکتای هر ستون را پیدا کرده و در قالب یک سری نگه میدارد (در مورد لامبدا اکسپرشن و یونیک قبلا توضیح داده شد، فقط قرار دادن axis = 1 موجب میشود حرکت اپلای به جای سطری، ستونی شود). سپس تعداد همه کلمات یک کتگوری را به دست می آورد و در قالب یک سری با کلید کتگوری و مقدار تعداد کلمات نگه میدارد. سپس adt_value یا همان آلفا در فرمول اصلی محاسبه میشود. علت اینکه این مقدار ثابت قرار داده نشد این بود که در صورت ثابت بودن، برای کتگوری با تعداد کلمات کمتر، وزن بیشتری پیدا میکرد، فلذا آلفا متناسب با مجموع تعداد کلمات قرار داده شد (ضریب تناسب با آزمون و خطا به دست آمده) . در مرحله آخر روی دیتافریم پاس داده شده فرمول ادیتیو اسموتینگ پیاده میشود (با استفاده از قابلیت های پانداز، همه خانه های دیتا فریم با یک مقدار ثابت جمع میشود، سپس همه سطر های دیتا فریم بر یک سطر تقسیم میشود که در واقع حاصل جمع سری مجموع کلمات و سری مقدار های یکتا ضرب در آلفا میباشد). در نهایت دیتا فریم اسموت شده را برگردانده میشود.  </p>

In [21]:
def make_adt_smooth(df : pd.DataFrame):
  final_df = df.copy()
  unique_vals = df.apply(lambda x : len(pd.unique(x)), axis=0)
  observ_cnt = df.sum(axis=0)
  adt_value = observ_cnt * 0.0001
  final_df = (final_df + adt_value).div(observ_cnt + (adt_value * unique_vals),axis=1)
  return final_df

<p dir="rtl" lang="fa" align="center">محاسبات را یک بار دیگر با ادیتیو اسموتینگ انجام داده و نتیجه را مشاهده میکنیم:</p>

In [22]:
smoothed_BoW = make_adt_smooth(BoW)
categories_list_adt = books_test_desc.apply(lambda x: cal_prob(x,smoothed_BoW,prior))
(categories_list_adt == books_test["categories"]).value_counts()

True     364
False     86
Name: count, dtype: int64

<p dir="rtl" lang="fa" align="center">نتیجه بسیار خوب است! تقریبا 80 درصد ورودی ها به درستی تشخیص داده شدند و با تقریب خوبی میتوان گفت به خواسته مطلوبمان رسیدیم.</p>

<h2>bonus content</h2>
<p dir="rtl" lang="fa" align="center">تبریک بابت تمام کردن بخش معمولی پروژه! وقت آن است سراغ محتوای امتیازی برویم.</p>

<h3>bonus part1: lemetize  and stem</h3>

<h4>bonus part1-1: lemetize</h4>

<p dir="rtl" lang="fa" align="center">کتابخانه هضم دو تابع مفید برای پری پراسس به نام stem و lemetize دارد که استم وظیفه حذف کردن پیشوند و پسوند اضافه کلمه بدون توجه به معنی کلمه و تابع لمتایز وظیفه دارد کلمه را به بن ماضی و مضارع خود تبدیل کند. ببینم هر تابع چقدر میتوان مفید یا مضر باشد.</p>

<p dir="rtl" lang="fa" align="center">تابع lem_text عمل لمتایز را روی کلمه مورد نظر انجام داده و از آنجایی که کتابخانه هضم بن ماضی و مضارع را با # از هم جدا میکند، باید این علامت را با اسپیس جای گذاری کرده و برگرداند. تابع clean_text_lem  نیز روی هر کلمه این تابع را اجرا میکند. بقیه اجزا مانند گذشته است.</p>

In [23]:
lem_obj = hz.Lemmatizer()
def lem_text(text : str) -> list:
  return lem_obj.lemmatize(text).replace('#',' ')

def clean_text_lem(text : str) -> str:
  text = norm_obj.normalize(text).replace("‌"," ")
  tokenized_text_list = token_obj.tokenize(text)
  semi_final_text = " ".join([lem_text(x) for x in tokenized_text_list if x not in words_to_delete])
  final_text = "".join([x for x in semi_final_text if x not in spec_chars])
  return final_text

<p dir="rtl" lang="fa" align="center">همه کار هایی که پیشتر انجام داده بودیم را دوباره با تابع پری پراسس جدیدمان انجام میدهیم.</p>

In [24]:
books_test = pd.read_csv("books_test.csv")
books_train = pd.read_csv("books_train.csv")

books_train["description"] = books_train["description"].apply(clean_text_lem)
books_test["description"] = books_test["description"].apply(clean_text_lem)

categories = list(books_train['categories'].unique())
category_filtered_list = make_cat_filt(categories, books_train)
BoW = make_BoW(category_filtered_list, categories)

books_test_desc = make_books_test_desc(books_test)

<h4>without additive smoothing</h4>

<p dir="rtl" lang="fa" align="center">حال دوباره تابع بدون ادیتیو اسموتینگ خود را روی این BoW جدید امتحان میکنیم</p>

In [25]:
prob_bow = make_prob_df(BoW)
categories_list_adt = books_test_desc.apply(lambda x: cal_prob(x,smoothed_BoW,prior))
(categories_list_noadt == books_test["categories"]).value_counts()

False    441
True       9
Name: count, dtype: int64

<p dir="rtl" lang="fa" align="center">تغییر خاصی در این بخش ایجاد نشد</p>

<h4>with additive smoothing</h4>

<p dir="rtl" lang="fa" align="center">و حالا ادیتیو اسموتینگ</p>

In [26]:
smoothed_BoW = make_adt_smooth(BoW)
categories_list_adt = books_test_desc.apply(lambda x: cal_prob(x,smoothed_BoW,prior))
(categories_list_adt == books_test["categories"]).value_counts()

True     359
False     91
Name: count, dtype: int64

<p dir="rtl" lang="fa" align="center">علت کمتر شدن این مقدار، این است که کلماتی که یکسان نیستند مانند اندیشمند و اندیشه و هم اندیش، هر سه تبدیل به بن خود یعنی اندیش میشوند و این موجب اختلال در محاسبه احتمال میشود. </p>

<h4>bonus part1-2: stem</h4>

<p dir="rtl" lang="fa" align="center">در این بخش میپردازیم به تابع استم. تابع stem_text وظیفه استم کردن استرینگ ورودی به آن را دارد. برای استم شدن هر کلمه، این  تابع را داخل لیست کلمات توکنایز شده بر هر کلمه اجرا میکنیم. بقیه قسمت ها مانند قبل میباشد.</p>

In [27]:
stem_obj = hz.Stemmer()
def stem_text(text : str) -> str:
  text = stem_obj.stem(text)
  return text

def clean_text_stem(text : str) -> str:
  text = norm_obj.normalize(text).replace("‌"," ")
  tokenized_text_list = token_obj.tokenize(text)
  semi_final_text = " ".join([stem_text(x) for x in tokenized_text_list if x not in words_to_delete])
  final_text = "".join([x for x in semi_final_text if x not in spec_chars])
  return final_text

In [28]:
books_test = pd.read_csv("books_test.csv")
books_train = pd.read_csv("books_train.csv")

books_train["description"] = books_train["description"].apply(clean_text_lem)
books_test["description"] = books_test["description"].apply(clean_text_lem)

categories = list(books_train['categories'].unique())
category_filtered_list = make_cat_filt(categories, books_train)
BoW = make_BoW(category_filtered_list, categories)

books_test_desc = make_books_test_desc(books_test)

<h4>without additive smoothing</h4>

<p dir="rtl" lang="fa" align="center">ببینیم چقدر وضع بهتر یا بدتر میشود</p>

In [29]:
prob_bow = make_prob_df(BoW)
categories_list_adt = books_test_desc.apply(lambda x: cal_prob(x,smoothed_BoW,prior))
(categories_list_noadt == books_test["categories"]).value_counts()

False    441
True       9
Name: count, dtype: int64

<h4>with additive smoothing</h4>

In [30]:
smoothed_BoW = make_adt_smooth(BoW)
categories_list_adt = books_test_desc.apply(lambda x: cal_prob(x,smoothed_BoW,prior))
(categories_list_adt == books_test["categories"]).value_counts()

True     359
False     91
Name: count, dtype: int64

<p dir="rtl" lang="fa" align="center">همانطور که مشاهده کردید، نتایج این بخش هیچ تفاوتی با بخش لمتایز نداشتند. چرا که این دو در واقع کاری شبیه به هم را انجام میدهند و تفاوت جزئی آنها موجب نمیشود تغییر چندانی در خروجی به دست بیاید</p>

<h3>bonus part2: stop words</h3>

<p dir="rtl" lang="fa" align="center">بخش مورد بحث بعدی استاپ وردز است که از طریق لیست آماده کتابخانه هضم و چک کردن آن در کنار لیست خودمان آن را در پری پراسس وارد میکنیم. استاپ وردز کلمات پرتکراری هستند که در جمله ها بسیار دیده میشوند ولی بودن یا نبودن آنها بار معنایی به کلمات BoW اصافه نمیکند، مانند است، بود شد و... . بقیه قسمت ها مانند قسمتهای قبل میباشند.</p>

In [31]:
stop_words = hz.stopwords_list()

def clean_text_sto(text : str) -> str:
  text = norm_obj.normalize(text).replace("‌"," ")
  tokenized_text_list = token_obj.tokenize(text)
  semi_final_text = " ".join([x for x in tokenized_text_list if x not in words_to_delete and x not in stop_words])
  final_text = "".join([x for x in semi_final_text if x not in spec_chars])
  return final_text

In [32]:
books_test = pd.read_csv("books_test.csv")
books_train = pd.read_csv("books_train.csv")

books_train["description"] = books_train["description"].apply(clean_text_sto)
books_test["description"] = books_test["description"].apply(clean_text_sto)

categories = list(books_train['categories'].unique())
category_filtered_list = make_cat_filt(categories, books_train)
BoW = make_BoW(category_filtered_list, categories)

books_test_desc = make_books_test_desc(books_test)

<h4>without additive smoothing</h4>

<p dir="rtl" lang="fa" align="center">ببینیم آیا استاپ وردز کمکی به دقت ما میکند؟</p>

In [33]:
prob_bow = make_prob_df(BoW)
categories_list_adt = books_test_desc.apply(lambda x: cal_prob(x,smoothed_BoW,prior))
(categories_list_noadt == books_test["categories"]).value_counts()

False    441
True       9
Name: count, dtype: int64

<p dir="rtl" lang="fa" align="center">در این بخش چیزی اضافه نمیشود</p>

<h4>with additive smoothing</h4>

In [34]:
smoothed_BoW = make_adt_smooth(BoW)
categories_list_adt = books_test_desc.apply(lambda x: cal_prob(x,smoothed_BoW,prior))
(categories_list_adt == books_test["categories"]).value_counts()

True     369
False     81
Name: count, dtype: int64

<p dir="rtl" lang="fa" align="center">و بالاخره با ادیتیو اسموتینگ، به دقت کد ما اضافه شد. یکی از عللی که کار های بخش امتیازی کمک چشمگیری در خروجی نکردند این بود که فایل شخصی words to delete تا حد خوبی برخی ازین موارد را پوشش میداد. از طرفی در مورد بخش بدون ادیتیو اسموتینگ خطا آنقدر بالا هست که تغییرات کوچک در پری پراسس تاثیر چندانی روی این بخش ها نداشته باشد. در کل بهترین نتیجه همانطور که شخصا انتظار داشتم، با استاپ وردز به دست آمد.</p>

<p dir="rtl" lang="fa" align="center">پاسخ به سوال نهایی امتیازی: توابع لمتایز و استم به ذاته توابع خوبی هستند، اما به دلایلی از جمله قاطی شدن بن های کلمات و اشتباه در استم کردن (به طور مثال رضوان را به رضو تبدیل میکند) بهتر است به تنهایی استفاده شوند و در ترکیب با یک لیست استاپ وردز استفاده نشوند، چرا که استاپ وردز علاوه بر افعال و اسامی، به تنهایی برخی کلمات و پیشوند ها و پسوند ها را حذف میکند و به این ترتیب ممکن است این توابع با انجام عمل بی مورد روی دیتا، درستی آن را کمتر کنند. نوع کاکرد آنها نیز پیشتر توضیح داده شد. در نهایت در صورتی که بخواهیم بین استاپ وردز و این توابع هضم یکی را انتخاب کنیم، بهتر است استاپ وردز انتخاب شود چرا که دقیق است و احتمال اشتباه در آن خیلی کم میباشد و از طرفی اورهد کمتری نسبت به این توابع کتابخانه هضم دارد.</p>